In [2]:
# !pip install numpy-financial
# !pip install yahoo-fin
# !pip install jproperties

## Imports

In [3]:
# Read stocks
import yfinance as yf

# To get analysts info
import yahoo_fin.stock_info as si

# Required to calculate net present value
import numpy_financial as npf

# For reading properties
from jproperties import Properties

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load properties

In [5]:
configs = Properties()

with open('/content/drive/MyDrive/Personal Projects/Stock-Analysis/yf_dcf.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Perpetual growth rates is read as a string; we need to convert it to float
PERP_GROWTH_RATE = float(configs.get('PERP_GROWTH_RATE').data)
WACC_RATE = float(configs.get('WACC_RATE').data)

## Constants

In [6]:
PROJECTION_YEARS = 5

## Get Growth Rate

In [14]:
growth_est_df = si.get_analysts_info(TICKER)['Growth Estimates']
growth_str = growth_est_df[growth_est_df['Growth Estimates'] == 'Next 5 Years (per annum)'][TICKER].iloc[0]
growth_rate = round(float(growth_str.rstrip('%')) / 100.0, 4)

print(growth_rate)

0.0563


## Future Free Cash Flow

In [15]:
# Using yfinance to get other data
ticker = yf.Ticker(TICKER)

# Free cash flow
free_cash_flow = ticker.cash_flow.loc['Free Cash Flow'][0]

ffcf = []
# Year 1
ffcf.append(free_cash_flow * (1 + growth_rate))

# Starting from year 2
for i in range(1, PROJECTION_YEARS):
    ffcf.append(ffcf[i-1] * (1 + growth_rate))
ffcf

[105190579200.0,
 111112808808.96,
 117368459944.90445,
 123976304239.80257,
 130956170168.50345]

## Calculate the Terminal Value

In [16]:
# forecast_fcf[-1] refers to the last year in the growth period
terminal_value = ffcf[-1] * (1 + PERP_GROWTH_RATE)/(WACC_RATE - PERP_GROWTH_RATE)

## Enterprise Valuation

In [21]:
# Add the terminal value to the last year
ffcf[-1] = ffcf[-1] + terminal_value

# Calcualte the enterprise_value using npv - add zero or else the method assumes
# first value as the initial investment
enterprise_value = npf.npv(WACC_RATE, [0] + ffcf)
# enterprise_value = sum(pv_ffcf)
print('${:,.2f}'.format(enterprise_value))

$4,172,969,387,491.40


## Equity Valuation

In [23]:
# Calculate Cash And Cash Equivalents
balance_sheet_df = ticker.balance_sheet
cash_and_equivalents = balance_sheet_df.loc['Cash And Cash Equivalents'][0]

# Current debt - only interested in the value
current_debt = balance_sheet_df.loc['Current Debt'][0]

# Long term debt - only interested in the value
long_term_debt = balance_sheet_df.loc['Long Term Debt'][0]

# Equity value -> enterprise_value + cash - total debt
equity_value = enterprise_value + cash_and_equivalents - (current_debt +  long_term_debt)
print('${:,.2f}'.format(equity_value))

$4,091,846,387,491.40


## Intrinsic Valuation

In [24]:
# Shares outstanding
shares_outstanding = ticker.info['sharesOutstanding']

# Current stock price
current_value = ticker.info['currentPrice']

# Intrinsic value
intrinsic_value = equity_value / shares_outstanding
'Current Price: ${:.2f} Intrinsic Value: ${:.2f}'.format(round(current_value,2), round(intrinsic_value,2)) + ' for ticker ' + TICKER

'Current Price: $195.89 Intrinsic Value: $263.09 for ticker AAPL'

In [25]:
upside = (intrinsic_value / current_value) - 1
'{:.2%}'.format(round(upside,4)) + ' for ticker ' + TICKER

'34.31% for ticker AAPL'

In [26]:
if intrinsic_value > current_value:
    print('BUY')
else:
    print('SELL')

BUY


## Enterprise Valuation - alternative

In [28]:
# This is another way of calculating PV of FFCF - sum of it should match the npv calculation of npf
pv_ffcf = []
for idx, x in enumerate(ffcf):
    pv_ffcf.append(x/(1 + WACC_RATE)**(idx+1))
# This is equivalent to enterprise_value value
print('${:,.2f}'.format(sum(pv_ffcf)))

$4,172,969,387,491.40
